# Лабораторная работа №1 "Градиентный спуск и его модификации"

## Задание

1. Градиентный спуск и его модификации
   - Выбрать [тестовые функции оптимизации](https://ru.wikipedia.org/wiki/Тестовые_функции_для_оптимизации) (2 шт)
   - Запрограммировать собственную реализацию классического градиентного спуска
   - Запрограммировать пайлайн тестирования алгоритма оптимизации
     - Визуализации функции и точки оптимума
     - Вычисление погрешности найденного решения в сравнение с аналитическим для нескольких запусков
     - Визуализации точки найденного решения (можно добавить анимацию на плюс балл)
   - Запрограммировать метод вычисления градиента
     - Передача функции градиента от пользователя
     - Символьное вычисление градиента (например с помощью [sympy](https://www.sympy.org/en/index.html)) (на доп балл)
     - Численная аппроксимация градиента (на доп балл)
   - Запрограммировать одну моментную модификацию и протестировать ее
   - Запрограммировать одну адаптивную модификацию и протестировать ее
   - Запрограммировать метод эфолюции темпа обучения и/или метод выбора начального приближения и протестировать их
   - `Will be unclocked afetr Lecture №5`

## Решение

В качестве тестовых функций возьмем: 
- **функция Розенброка** 

1. $ f(x) = \sum _ {i=1} ^ {n-1} [100(x_{i+1}-x_{i}^{2})^2+(x_i-1)^2]$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Rosenbrock%27s_function_in_3D.pdf/page1-640px-Rosenbrock%27s_function_in_3D.pdf.jpg?uselang=ru)

2. Минимум: $ f(1_0, 1_1, ..., 1_n)=0 $

3. Метод поиска: $-\infty \leq x_i \leq \infty, 1 \leq i \leq n$

- **функция Била**

1. $ f(x,y) = (1.5-x+xy)^2+(2.25-x+xy^2)^2+(2.625-x+xy^3)^2 $

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Beale%27s_function.pdf/page1-640px-Beale%27s_function.pdf.jpg?uselang=ru)

2. Минимум: $ f(3, 0.5) = 0 $

3. Метод поиска: $-4.5 \leq x,y \leq 4.5$

In [1]:
import numpy as np
import sympy as sp
import plotly.graph_objects as go
import plotly.colors as colors

Напишем вычисление выбранных тестовых функций. Для функции Розенброка реализуем вариант функции от двух переменных.

In [31]:
def rosenbrock(points):
    """
    Функция Розенброка для массива точек [[x0, y0], [x1, y1], ...].
    
    Parameters:
        points (np.ndarray): Массив размерности (N, 2), где каждая строка - это [x, y].

    Returns:
        np.ndarray: Значения функции Розенброка для каждой пары [x, y].
    """
    x = points[:, 0]
    y = points[:, 1]
    return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

def beale(points):
    """
    Функция Била для массива точек [[x0, y0], [x1, y1], ...].

    Parameters:
        points (np.ndarray): Массив размерности (N, 2), где каждая строка - это [x, y].

    Returns:
        np.ndarray: Значения функции Била для каждой пары [x, y].
    """
    x = points[:, 0]
    y = points[:, 1]
    return (
        (1.5 - x + x * y) ** 2 +
        (2.25 - x + x * y ** 2) ** 2 +
        (2.625 - x + x * y ** 3) ** 2
    )

Реализуем классический алгоритм градиентного спуска. На данном этапе будем подавать при вызове функцию для подсчета градиента. Позже, после выполнения задания по численной аппроксимации, сосздадим еще одну имплементацию, которая уже будет использовать приблежнные значения градиента.

In [3]:
def gradient_descent(func, grad_func, initial_points, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """
    Классический градиентный спуск для оптимизации функции.

    Parameters:
        func (callable): Целевая функция, которую нужно минимизировать.
        grad_func (callable): Функция, возвращающая градиенты целевой функции.
        initial_points (np.ndarray): Начальные точки размерности (N, 2).
        learning_rate (float): Скорость обучения (шаг градиентного спуска).
        max_iter (int): Максимальное количество итераций.
        tol (float): Критерий остановки по норме градиента.

    Returns:
        tuple: (np.ndarray, list): Оптимизированные точки размерности (N, 2) и список траекторий точек.
    """
    points = initial_points.copy()
    trajectory = [initial_points.copy()]
    for i in range(max_iter):
        gradients = grad_func(points)
        norm = np.linalg.norm(gradients, axis=1).max()
        if norm < tol:
            break
        points -= learning_rate * gradients
        trajectory.append(points.copy())
    return points, trajectory


Поскольку метод градиентного спуска реализован пока что через подсчет по готовой функции градиента, воспользуемся библиотекой sympy для нахождения функции градиента.

In [4]:
def compute_gradient(func):
    # Запрос функции от пользователя
    expr_str = func
    
    # Парсим введенную строку как выражение sympy
    expr = sp.sympify(expr_str)
    
    # Определяем переменные
    variables = list(expr.free_symbols)  # Автоматически находим все переменные
    
    # Вычисляем градиент
    gradient = [sp.diff(expr, var) for var in variables]
    
    # Вывод результата
    for var, partial_derivative in zip(variables, gradient):
        print(f"∂/∂{var}: {partial_derivative}")

In [5]:
print("Градиент функции Розенброка:")
compute_gradient('100 * (y - x**2)**2 + (x - 1)**2')
print("Градиент функции Била:")
compute_gradient('((1.5 - x + x * y)**2 + (2.25 - x + x * y**2)**2 + (2.625 - x + x * y**3)**2)')

Градиент функции Розенброка:
∂/∂y: -200*x**2 + 200*y
∂/∂x: -400*x*(-x**2 + y) + 2*x - 2
Градиент функции Била:
∂/∂y: 15.75*x*y**2*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1) + 9.0*x*y*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 3.0*x*(0.666666666666667*x*y - 0.666666666666667*x + 1)
∂/∂x: 2.25*(1.33333333333333*y - 1.33333333333333)*(0.666666666666667*x*y - 0.666666666666667*x + 1) + 5.0625*(0.888888888888889*y**2 - 0.888888888888889)*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 6.890625*(0.761904761904762*y**3 - 0.761904761904762)*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1)


Запишем полученные градиенты в функции:

In [6]:
def rosenbrock_gradient(points):
    """
    Градиенты функции Розенброка для массива точек [[x0, y0], [x1, y1], ...].

    Parameters:
        points (np.ndarray): Массив размерности (N, 2), где каждая строка - это [x, y].

    Returns:
        np.ndarray: Градиенты функции Розенброка для каждой пары [x, y].
    """
    x = points[:, 0]
    y = points[:, 1]
    grad_x = -2 * (1 - x) - 400 * x * (y - x ** 2)
    grad_y = 200 * (y - x ** 2)
    return np.stack([grad_x, grad_y], axis=1)

def beale_gradient(points):
    """
    Градиенты функции Била для массива точек [[x0, y0], [x1, y1], ...].

    Parameters:
        points (np.ndarray): Массив размерности (N, 2), где каждая строка - это [x, y].

    Returns:
        np.ndarray: Градиенты функции Била для каждой пары [x, y].
    """
    x = points[:, 0]
    y = points[:, 1]
    grad_x = (
        2.25 * (1.33333333333333 * y - 1.33333333333333) * (0.666666666666667 * x * y - 0.666666666666667 * x + 1) +
        5.0625 * (0.888888888888889 * y ** 2 - 0.888888888888889) * (0.444444444444444 * x * y ** 2 - 0.444444444444444 * x + 1) +
        6.890625 * (0.761904761904762 * y ** 3 - 0.761904761904762) * (0.380952380952381 * x * y ** 3 - 0.380952380952381 * x + 1)
    )
    grad_y = (
        15.75 * x * y ** 2 * (0.380952380952381 * x * y ** 3 - 0.380952380952381 * x + 1) +
        9.0 * x * y * (0.444444444444444 * x * y ** 2 - 0.444444444444444 * x + 1) +
        3.0 * x * (0.666666666666667 * x * y - 0.666666666666667 * x + 1)
    )
    return np.stack([grad_x, grad_y], axis=1)



Напишем пайплайн визуализации и подсчета ошибки

In [7]:
def visualize_3d(func, trajectories, x_range, y_range, title="3D Visualization"):
    """
    Визуализация функции и траектории градиентного спуска в 3D с использованием Plotly.

    Parameters:
        func (callable): Целевая функция для визуализации.
        trajectory (list): Список точек траектории [[x0, y0], [x1, y1], ...].
        x_range (tuple): Диапазон значений для оси X (min, max).
        y_range (tuple): Диапазон значений для оси Y (min, max).
        title (str): Заголовок графика.
    """
    x = np.linspace(x_range[0], x_range[1], 100)
    y = np.linspace(y_range[0], y_range[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = func(np.column_stack([X.ravel(), Y.ravel()])).reshape(X.shape)
    
    fig = go.Figure()
    
    # Поверхность функции
    fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.7))
    
    for i, trajectory in enumerate(trajectories):
        trajectory = np.array(trajectory).reshape(len(trajectory), 2)
        # Траектория спуска
        fig.add_trace(go.Scatter3d(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            z=func(trajectory),
            mode='lines+markers',
            marker=dict(size=5, color=colors.qualitative.Plotly[i]),
            line=dict(color=colors.qualitative.Plotly[i], width=2)
        ))

    fig.update_layout(title=title, scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ))
    
    fig.show()

Напишем функцию вычисления погрешности

In [33]:
def compute_error(func, optimized_points, true_minimum):
    """
    Вычисляет погрешность между найденными минимумами и аналитическим минимумом.

    Parameters:
        func (callable): Целевая функция.
        optimized_points (np.ndarray): Найденные точки минимума размерности (N, 2).
        true_minimum (np.ndarray): Аналитический минимум (1, 2).

    Returns:
        tuple: Среднеквадратичная ошибка координат и значений функции.
    """
    errors_points = np.linalg.norm(optimized_points - true_minimum, axis=1)
    errors_values = func(optimized_points) - func(true_minimum)
    return np.mean(errors_points ** 2), np.mean(errors_values ** 2)

Рассмотрим работу алгоритма для функции Розенброка

In [52]:
trajectories = []
result = []
points = [[0.0, 3.0], [2.0, -1.0], [-2.0, -1.0]]
for point in points:
    # Пример использования градиентного спуска для функции Розенброка
    initial_points = np.array([point])
    optimized_points, trajectory = gradient_descent(
        func=rosenbrock,
        grad_func=rosenbrock_gradient,
        initial_points=initial_points,
        learning_rate=0.001,
        max_iter=10000,
        tol=1e-8
    )
    trajectories += [trajectory]
    result += [optimized_points]

In [ ]:
visualize_3d(
    func=rosenbrock,
    trajectories=trajectories,
    x_range=(-2, 2),
    y_range=(-1, 3),
    title="Функция Розенброка"
)

*Поскольку github не отображает анимированный plotly, я сохранил изображение*

![rosenbrock](./data/rosenbrock.png)

In [53]:
errors_points, errors_values = compute_error(rosenbrock, np.array(result).reshape(len(result), 2), np.array([[1.0, 1.0]]))
print(f"MSE для координат минимума: {errors_points}")
print(f"MSE для найденного минимума: {errors_values}")

MSE для координат минимума: 0.00013828855649871994
MSE для найденного минимума: 1.2448700002868934e-09


Рассмотрим работу алгоритма для функции Била

In [55]:
trajectories = []
result = []
points = [[3.0, 3.0], [3.0, -3.0]]
for point in points:
    # Пример использования градиентного спуска для функции Била
    initial_points = np.array([point])
    optimized_points, trajectory = gradient_descent(
        func=beale,
        grad_func=beale_gradient,
        initial_points=initial_points,
        learning_rate=0.00001,
        max_iter=100000,
        tol=1e-8
    )
    trajectories += [trajectory]
    result += [optimized_points]

In [ ]:
visualize_3d(
    func=beale,
    trajectories=trajectories,
    x_range=(-3, 3),
    y_range=(-3, 3),
    title="Функция Била"
)

*Поскольку github не отображает анимированный plotly, я сохранил изображение*

![rosenbrock](./data/beale.png)

In [36]:
errors_points, errors_values = compute_error(beale, np.array(result).reshape(len(result), 2), np.array([[3.0, 0.5]]))
print(f"MSE для координат минимума: {errors_points}")
print(f"MSE для найденного минимума: {errors_values}")

MSE для координат минимума: 0.14163942064273022
MSE для найденного минимума: 0.0018572580734444357


Высокая ошибка для функции Била объясняется долгой сходимостью. Если поставить больше итераций, то ошибка будет меньше. На визуализации видно, что спуск идет ровно в минимум.

Теперь реализуем моментную и адаптивную модификации

In [37]:
def momentum_gradient_descent(func, grad_func, initial_points, learning_rate=0.01, momentum=0.9, max_iter=1000, tol=1e-6):
    """
    Градиентный спуск с моментумом для оптимизации функции.

    Parameters:
        func (callable): Целевая функция, которую нужно минимизировать.
        grad_func (callable): Функция, возвращающая градиенты целевой функции.
        initial_points (np.ndarray): Начальные точки размерности (N, 2).
        learning_rate (float): Скорость обучения (шаг градиентного спуска).
        momentum (float): Коэффициент момента.
        max_iter (int): Максимальное количество итераций.
        tol (float): Критерий остановки по норме градиента.

    Returns:
        tuple: (np.ndarray, list): Оптимизированные точки размерности (N, 2) и список траекторий точек.
    """
    points = initial_points.copy()
    velocity = np.zeros_like(points)
    trajectory = [points.copy()]
    for i in range(max_iter):
        gradients = grad_func(points)
        norm = np.linalg.norm(gradients, axis=1).max()
        if norm < tol:
            break
        velocity = momentum * velocity - learning_rate * gradients
        points += velocity
        trajectory.append(points.copy())
    return points, trajectory

def adaptive_gradient_descent(func, grad_func, initial_points, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """
    Адаптивный градиентный спуск для оптимизации функции.

    Parameters:
        func (callable): Целевая функция, которую нужно минимизировать.
        grad_func (callable): Функция, возвращающая градиенты целевой функции.
        initial_points (np.ndarray): Начальные точки размерности (N, 2).
        learning_rate (float): Начальная скорость обучения.
        max_iter (int): Максимальное количество итераций.
        tol (float): Критерий остановки по норме градиента.

    Returns:
        tuple: (np.ndarray, list): Оптимизированные точки размерности (N, 2) и список траекторий точек.
    """
    points = initial_points.copy()
    cache = np.zeros_like(points)
    trajectory = [points.copy()]
    epsilon = 1e-8
    for i in range(max_iter):
        gradients = grad_func(points)
        norm = np.linalg.norm(gradients, axis=1).max()
        if norm < tol:
            break
        cache += gradients ** 2
        adjusted_gradients = gradients / (np.sqrt(cache) + epsilon)
        points -= learning_rate * adjusted_gradients
        trajectory.append(points.copy())
    return points, trajectory